#### CMSC 462 Assignment 5 - Hamin Han

The following command resets the environment

In [1]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Import mongo

In [2]:
from pymongo import MongoClient
import pprint
import pymongo
import pandas as pd
import datetime
from bson.code import Code

Connect to  MongoDB server

In [3]:
try:
    client = MongoClient("mongodb://localhost:27017/")
    client.drop_database("Mongo_database")
except Exception as e:
    print(e)

Create database

In [4]:
try:
    db=client.get_database("Mongo_database")
except Exception as e:
    print(e)
 

Read CSV files and insert data into MongoDB

In [5]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
tags_df = pd.read_csv('tags.csv')

Insert data into respective collections

In [6]:
db.movies.insert_many(movies_df.to_dict('records'))
db.ratings.insert_many(ratings_df.to_dict('records'))
db.tags.insert_many(tags_df.to_dict('records'))

InsertManyResult([ObjectId('656c2055b74c54baaf168938'), ObjectId('656c2055b74c54baaf168939'), ObjectId('656c2055b74c54baaf16893a'), ObjectId('656c2055b74c54baaf16893b'), ObjectId('656c2055b74c54baaf16893c'), ObjectId('656c2055b74c54baaf16893d'), ObjectId('656c2055b74c54baaf16893e'), ObjectId('656c2055b74c54baaf16893f'), ObjectId('656c2055b74c54baaf168940'), ObjectId('656c2055b74c54baaf168941'), ObjectId('656c2055b74c54baaf168942'), ObjectId('656c2055b74c54baaf168943'), ObjectId('656c2055b74c54baaf168944'), ObjectId('656c2055b74c54baaf168945'), ObjectId('656c2055b74c54baaf168946'), ObjectId('656c2055b74c54baaf168947'), ObjectId('656c2055b74c54baaf168948'), ObjectId('656c2055b74c54baaf168949'), ObjectId('656c2055b74c54baaf16894a'), ObjectId('656c2055b74c54baaf16894b'), ObjectId('656c2055b74c54baaf16894c'), ObjectId('656c2055b74c54baaf16894d'), ObjectId('656c2055b74c54baaf16894e'), ObjectId('656c2055b74c54baaf16894f'), ObjectId('656c2055b74c54baaf168950'), ObjectId('656c2055b74c54baaf1689

Add five movies to the "movies" collection

In [7]:
movie_list = [
    {"movieId": 200000, "title": "The Notebook", "genres": "Romance"},
    {"movieId": 2000001, "title": "Titanic", "genres": "Romance"},
    {"movieId": 2000002, "title": "10 things I Hate About You", "genres": "Romance"},
    {"movieId": 2000003, "title": "100 Days of Summer", "genres": "Romance"},
    {"movieId": 2000004, "title": "Crazy Stupid Love", "genres": "Romance"}
]

db.movies.insert_many(movie_list)

InsertManyResult([ObjectId('656c22abb74c54baaf16979b'), ObjectId('656c22abb74c54baaf16979c'), ObjectId('656c22abb74c54baaf16979d'), ObjectId('656c22abb74c54baaf16979e'), ObjectId('656c22abb74c54baaf16979f')], acknowledged=True)

Add ratings and tags for the added movies 

In [ ]:
user_id = "9595"  # Personal unique user_id

for movie in movie_list:
    movie_id = movie["movieId"]
    
    # Add ratings
    db.ratings.insert_one({"userId": user_id, "movieId": movie_id, "rating": 5})
    
    # Add tags
    db.tags.insert_one({"userId": user_id, "movieId": movie_id, "tag": "romantic"})


Use Aggregation Pipeline for Analysis

In [ ]:
pipeline = [
    # a. Number of movies released per year
    {"$group": {"_id": "$year", "movies_per_year": {"$sum": 1}}},

    # b. Number of movies per genre
    {"$unwind": "$genres"},
    {"$group": {"_id": "$genres", "movies_per_genre": {"$sum": 1}}},

    # c. Number of movies per rating
    {"$group": {"_id": "$rating", "movies_per_rating": {"$sum": 1}}},

    # d. Number of movies tagged
    {"$group": {"_id": "$movieId", "movies_tagged": {"$sum": 1}}},

    # e. Most popular tag
    {"$group": {"_id": "$tag", "tag_count": {"$sum": 1}}},
    {"$sort": {"tag_count": -1}},
    {"$limit": 1}
]

result = list(db.movies.aggregate(pipeline))
print(result)